In [19]:
import numpy as np

In [20]:
data2 = "1000001001000000111001101101101011000010111001001110100001000000111000001101000011011110110111001100101001011000010000001101100011010010110101101100101001000000111100101101111011101010111001000100000011011000110000101110000011101000110111101110000001000000110111101110010001000000110010001100101011100110110101101110100011011110111000000100000011000110110111101101101011100000111010101110100011001010111001000101100001000000110001101100001011011100"
data3 = "11010000001011100111"
data4 = "00101111110100011000"
data5 = "ton message est sage, simple et sagace"
data6 = "AABABBBABAABABBBABBABB"
data7 = 'LICENCE IRM A LA FST'
data8 = "BAABABBBAABBBBAA"
data9 = "1001011100011100101"
alphabet1 = "abcdegilmnoprstu ,"
alphabet2 = ['a', 'b', 'c', 'd', 'e', 'g', 'i', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', ' ', ',']
alphabet3 = [chr(i) for i in range(256)]
alphabet5 = ['A', 'B']
alphabet6 = ['0', '1']

#compressed = [14, 10, 9, 16, 8, 4, 13, 13, 0, 5, 4, 16, 23, 14, 16, 25, 27, 17, 32, 6, 8, 11, 7, 28, 4, 31, 33, 0, 2, 4]

with open("../../Textes/file2.txt", 'r') as fichier:
  text = fichier.read()

In [21]:
def compress_lzw(data, alphabet):
    dictionary = {}
    result = []
    current_code = len(alphabet)

    # Initialiser le dictionnaire avec les caractères de l'alphabet spécifié
    for i, char in enumerate(alphabet):
        dictionary[char] = i

    prefix = ''
    for char in data:
        new_entry = prefix + char
        if new_entry in dictionary:
            prefix = new_entry
        else:
            result.append(dictionary[prefix])
            dictionary[new_entry] = current_code
            current_code += 1
            prefix = char

    # Gérer le dernier préfixe
    if prefix:
        result.append(dictionary[prefix])

    return result

In [22]:
def decompress_lzw(compressed_data, alphabet):
    result = []
    dictionary = {}
    current_code = len(alphabet)

    # Initialiser le dictionnaire avec les caractères de l'alphabet spécifié
    for i, char in enumerate(alphabet):
        dictionary[i] = char

    old_entry = dictionary[compressed_data[0]]
    result.append(old_entry)
    for new_entry in compressed_data[1:]:
        if new_entry in dictionary:
            entry = dictionary[new_entry]
        elif new_entry == current_code:
            entry = old_entry + old_entry[0]
        else:
            raise ValueError("Mauvaise séquence compressée")

        result.append(entry)

        # Utilisez le même dictionnaire pour la décompression
        dictionary[current_code] = old_entry + entry[0]
        current_code += 1
        old_entry = entry

    return ''.join(result)

In [23]:
compressed = compress_lzw(data9, alphabet6)

In [24]:
decompressed = decompress_lzw(compressed, alphabet6)

In [25]:
data9 == decompressed

True

In [26]:
compressed

[1, 0, 0, 2, 1, 6, 3, 4, 7, 4, 4]

In [27]:
comp = [97, 256, 98, 98, 97, 259, 259, 257, 260, 98]

In [28]:
decompress_lzw(comp, alphabet3)

'aaabbababaaababb'

In [29]:
def taux_compression(data, compressed_data):
    maxi = max(compressed_data)
    bit_max = int(np.log2(maxi)) + 1
    taux = 1 - ((len(compressed_data) * bit_max) / (len(data) * 8))
    return taux

In [30]:
taux_compression(data6, compressed)

0.8125

In [31]:
def text_to_binary(text):
    # Convertir le texte en bytes en utilisant l'encodage Python par défaut (UTF-8)
    encoded_text = text.encode()
    
    # Convertir chaque byte en sa représentation binaire
    binary_sequence = ''.join(format(byte, '08b') for byte in encoded_text)
    
    return binary_sequence

In [32]:
def binary_to_text(binary_sequence):
    # Diviser la séquence binaire en octets de 8 bits
    bytes_list = [binary_sequence[i:i+8] for i in range(0, len(binary_sequence), 8)]
    
    # Convertir chaque octet en caractère en utilisant l'encodage UTF-8
    decoded_text = bytes(int(byte, 2) for byte in bytes_list).decode('utf-8').rstrip('\x00')
    
    return decoded_text

In [33]:
def lzw_to_text(data, alphabet):
  binary_sequence = text_to_binary(data)
  encoded_text = compress_lzw(binary_sequence, alphabet)
  return encoded_text

In [34]:
def text_to_lzw(encoded, alphabet):
  decompressed = decompress_lzw(encoded, alphabet)
  original_text = binary_to_text(decompressed)
  return original_text

In [35]:
encoded = lzw_to_text(text, alphabet3)

In [36]:
decoded = text_to_lzw(encoded, alphabet3)

In [37]:
text == decoded

True

In [38]:
taux_compression(text, encoded)

0.1337720636304206